In [182]:
import load_files as lf
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [3]:
rain_data = lf.get_rain("D:/waterschap-aa-en-maas_sewage_2019/sewer_data/rain_timeseries")
rain_prediction = lf.get_rain_prediction("D:/waterschap-aa-en-maas_sewage_2019/sewer_data/rain_grid_prediction/knmi.harmonie_2018-01-01_2019-08-29")
data = lf.sdf("D:/waterschap-aa-en-maas_sewage_2019/sewer_model/aa-en-maas_sewer_shp")

In [5]:
rain_prediction[0]

,pred,start,end
0,2018-01-01,2018-01-02 00:00:00,2018-01-02 01:00:00
1,2018-01-01,2018-01-02 00:00:00,2018-01-02 01:00:00
2,2018-01-01,2018-01-02 01:00:00,2018-01-02 02:00:00
3,2018-01-01,2018-01-02 02:00:00,2018-01-02 03:00:00
4,2018-01-01,2018-01-02 03:00:00,2018-01-02 04:00:00
5,2018-01-01,2018-01-02 03:00:00,2018-01-02 04:00:00
6,2018-01-01,2018-01-02 04:00:00,2018-01-02 05:00:00
7,2018-01-01,2018-01-02 05:00:00,2018-01-02 06:00:00
8,2018-01-01,2018-01-02 06:00:00,2018-01-02 07:00:00
9,2018-01-01,2018-01-02 07:00:00,2018-01-02 08:00:00


In [276]:
def cell_index(x, y):
    x_space = np.linspace(start = -0.0185, stop = -0.0185 + 299*0.037, num = 300)
    y_space = np.linspace(start = 48.9885, stop = 48.9885 + 299*0.023, num = 300)[::-1]
    
    x_out = np.where(~(x >= x_space))[0][0]
    y_out = np.where(~(y <= y_space))[0][0]
    
    return x_out, y_out

vec_cell_index = np.vectorize(cell_index)


class pred_to_rain:
    def __init__(self, rain_data, rain_prediction, area_data):
        if rain_data["Start"].dtype != "<M8[ns]":
            rain_data["Start"] = pd.to_datetime(rain_data["Start"])
        
        rain_data.sort_values("Start", inplace=True)
        rain_data.reset_index(drop=True, inplace=True)
        
        # ADD COLUMN OF ROW/COLUMN INDEX IN PREDICTION GRID
        area_data["x"] = area_data["geometry"].to_crs({'init': 'epsg:4326'}).centroid.x
        area_data["y"] = area_data["geometry"].to_crs({'init': 'epsg:4326'}).centroid.y
        
        indices = vec_cell_index(area_data["x"], area_data["y"])
        indices = list(zip(indices[0], indices[1]))
        
        area_data["loc"] = indices
        
        # 
        rain_data_time_span = rain_data["Start"].diff(1).apply(lambda i: i.total_seconds())
        rain_data_times = rain_data["Start"].apply(lambda i: i.replace(minute=0))
        base_time = rain_data_times[0]
        rain_data_int_times = rain_data_times.apply(lambda i: int((i - base_time).total_seconds()))
        
        date_to_index = pd.Series(range(len(rain_prediction[0])),
                                  index = rain_prediction[0]["start"]\
                                  .apply(lambda i: int((i - base_time).total_seconds())))
        
        grid_layers = date_to_index.loc[date_to_index.index.intersection(int_times)]
        grid_layers = grid_layers.loc[~grid_layers.index.duplicated(keep='first')]
        
        # ADD DATA TO CLASS
        self.rain_data = rain_data
        self.rain_prediction = rain_prediction
        self.area_data = area_data
        self.grid_layers = grid_layers
        self.base_time = base_time

    def exact(self):
        rain_values = self.area_data["loc"].apply(lambda i: pd.Series(rain_prediction[1][self.grid_layers, i[0], i[1]]))
        rain_values.index = self.area_data["area_name"]
        rain_values = rain_values.T
        rain_values["Start"] = pd.Series(self.grid_layers.index)\
                                         .apply(lambda i: self.base_time + datetime.timedelta(seconds = i))
        rain_values["End"] = rain_values["Start"].apply(lambda i: i + datetime.timedelta(hours = 1))
        
        return rain_values
    
    def estimated(self):
        pass

In [277]:
haha = pred_to_rain(rain_data, rain_prediction, data.area_data)

In [278]:
x = haha.exact()

In [279]:
x

area_name,Beijerd en 't Riet,30. Akkers I,gebied E; Overlaet E,Geb. 8 Orthenpoort,de Brand,Westerbroek,gebied C1; Hondberg-noord,Moerboom,Spoorstraat,geb. 6c Carolushof,...,08. Stiphout,Wildsedijk,Langenboom,Deursen,Dennenburg,RG Slievenpark,Coudevoort,Bolle Akker,Start,End
0,0.258789,0.003906,0.004883,0.001953,0.002930,0.000000,0.004883,0.023438,0.051758,0.001953,...,0.003906,0.011719,0.127930,0.280273,0.280273,0.132812,0.000000,0.211914,2018-01-02 00:00:00,2018-01-02 01:00:00
1,0.508789,0.164062,0.023438,0.027344,0.030273,0.055664,0.023438,0.434570,0.379883,0.027344,...,0.242188,0.028320,0.213867,0.049805,0.049805,0.029297,0.055664,0.316406,2018-01-02 01:00:00,2018-01-02 02:00:00
2,0.002930,0.130859,0.013672,0.022461,0.016602,0.003906,0.013672,0.009766,0.000000,0.022461,...,0.139648,0.006836,0.028320,0.146484,0.146484,0.373047,0.003906,0.241211,2018-01-02 02:00:00,2018-01-02 03:00:00
3,0.332406,0.156625,0.023813,-0.000601,0.019907,0.064828,0.023813,0.105844,0.352914,-0.000601,...,0.148813,0.037485,0.144907,0.832407,0.832407,0.086313,0.064828,0.169321,2018-01-02 03:00:00,2018-01-02 04:00:00
4,0.330350,0.005155,0.009061,0.026639,0.005155,0.001249,0.009061,0.048124,0.931912,0.026639,...,0.006132,0.034452,0.110624,0.722929,0.722929,0.056913,0.001249,0.030546,2018-01-02 04:00:00,2018-01-02 05:00:00
5,0.013672,0.000000,0.000977,0.000000,0.000977,0.001953,0.000977,0.081055,0.077148,0.000000,...,0.000000,0.000000,0.014648,0.001953,0.001953,0.000000,0.001953,0.000000,2018-01-02 05:00:00,2018-01-02 06:00:00
6,0.000000,0.000000,0.000977,0.007812,0.001953,0.000000,0.000977,0.000000,0.000977,0.007812,...,0.000000,0.001953,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2018-01-02 06:00:00,2018-01-02 07:00:00
7,0.114258,0.001953,0.004883,0.022461,0.012695,0.015625,0.004883,0.007812,0.203125,0.022461,...,0.000000,0.005859,0.013672,0.005859,0.005859,0.009766,0.015625,0.000977,2018-01-02 07:00:00,2018-01-02 08:00:00
8,0.077148,0.070312,0.004883,0.004883,0.000977,0.001953,0.004883,0.167969,0.018555,0.004883,...,0.055664,0.078125,0.226562,0.213867,0.213867,0.055664,0.001953,0.031250,2018-01-02 08:00:00,2018-01-02 09:00:00
9,0.036133,0.066406,0.000000,0.000000,0.000000,0.000000,0.000000,0.078125,0.148438,0.000000,...,0.017578,0.031250,0.007812,0.008789,0.008789,0.139648,0.000000,0.001953,2018-01-02 09:00:00,2018-01-02 10:00:00


In [294]:
hourly_rd = rain_data.iloc[:,3:].groupby(rain_data["Start"].apply(lambda i: i.replace(minute=0))).sum()

In [295]:
hourly_rd

,Lekerstraat,11. Speelweide/Vlinkert,Rips,Handel,14. Koolhof,13. Heiakker,Vlierden,Helenaveen,Heusden,Liender/Loverbosch-noord,...,Ooivaarsrijt,de Brand,Bitswijk,de Geer-zuid,De Hoeven(Haarsteeg),11+19 Grevekeur,CHV-terrein,Geb. 12 Vughterpoort,Heusdenseweg(Haarsteeg),Bosscheweg
Start,,,,,,,,,,,,,,,,,,,,,
2017-12-31 23:00:00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2018-01-01 00:00:00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0124,0.0000,0.0000,0.0000,0.0000,0.0000
2018-01-01 01:00:00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0244,0.0082,0.0000,0.0789,0.0489,0.0000,0.0000,0.0820,0.0000
2018-01-01 02:00:00,0.0000,0.0223,0.0000,0.0000,0.0000,0.0000,0.0000,0.1636,0.1107,0.0012,...,0.0186,0.0932,0.0199,0.0689,0.0186,0.0373,0.0351,0.0609,0.0205,0.0000
2018-01-01 03:00:00,0.0000,0.0000,0.0000,0.0000,0.0477,0.0049,0.0032,0.0000,0.1122,0.1388,...,0.1992,0.0509,0.0264,0.0984,0.0036,0.0269,0.0110,0.0323,0.0176,0.0000
2018-01-01 04:00:00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0197,0.1833,0.0000,0.0231,0.0672,0.0000,0.0000,0.0000
2018-01-01 05:00:00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.2495,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2018-01-01 06:00:00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2018-01-01 07:00:00,0.2993,0.0875,0.0000,0.1210,0.0000,0.0000,0.0818,0.0000,0.0016,0.1241,...,0.0186,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [301]:
x.set_index("Start").loc[:, hourly_rd.columns]

area_name,Lekerstraat,Lekerstraat,11. Speelweide/Vlinkert,Rips,Handel,14. Koolhof,13. Heiakker,13. Heiakker,Vlierden,Helenaveen,...,de Brand,Bitswijk,de Geer-zuid,De Hoeven(Haarsteeg),11+19 Grevekeur,CHV-terrein,Geb. 12 Vughterpoort,Heusdenseweg(Haarsteeg),Bosscheweg,Bosscheweg
Start,,,,,,,,,,,,,,,,,,,,,
2018-01-02 00:00:00,0.006836,0.006836,0.211914,0.081055,0.056641,0.362305,0.362305,0.362305,0.309570,0.336914,...,0.002930,0.003906,0.133789,0.001953,0.000000,0.000000,0.001953,0.001953,0.000000,0.000977
2018-01-02 01:00:00,0.442383,0.442383,0.316406,1.116211,0.351562,0.656250,0.656250,0.656250,0.237305,0.367188,...,0.030273,0.491211,0.046875,0.045898,0.116211,0.488281,0.027344,0.045898,0.061523,0.044922
2018-01-02 02:00:00,0.214844,0.214844,0.241211,0.048829,0.161133,0.653320,0.653320,0.653320,0.799805,0.286133,...,0.016602,0.136719,0.001953,0.008789,0.007812,0.000000,0.022461,0.008789,0.004883,0.021484
2018-01-02 03:00:00,0.058969,0.058969,0.169321,0.087289,0.141000,0.067758,0.067758,0.067758,0.072641,0.044321,...,0.019907,0.029672,0.023813,0.039438,0.039438,0.031625,-0.000601,0.039438,0.057016,0.015024
2018-01-02 04:00:00,0.006132,0.006132,0.030546,0.052030,0.049100,0.010038,0.010038,0.010038,0.022733,0.359647,...,0.005155,0.150663,0.040312,0.004179,0.005155,0.015898,0.026639,0.004179,0.005155,0.004179
2018-01-02 05:00:00,0.000000,0.000000,0.000000,0.005859,0.003906,0.000977,0.000977,0.000977,0.000000,0.173828,...,0.000977,0.000977,0.003906,0.000000,0.000977,0.000000,0.000000,0.000000,0.000000,0.001953
2018-01-02 06:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.001953,0.000000,0.000000,0.010742,0.000977,0.000000,0.007812,0.010742,0.009766,0.001953
2018-01-02 07:00:00,0.002930,0.002930,0.000977,0.008789,0.001953,0.005859,0.005859,0.005859,0.003906,0.016602,...,0.012695,0.000977,0.002930,0.096680,0.013672,0.001953,0.022461,0.096680,0.135742,0.019531
2018-01-02 08:00:00,0.029297,0.029297,0.031250,0.004883,0.021484,0.020508,0.020508,0.020508,0.035156,0.014648,...,0.000977,0.099609,0.288086,0.003906,0.025391,0.011719,0.004883,0.003906,0.017578,0.002930


In [304]:
x.loc[:, "Bosscheweg"]

area_name,Bosscheweg,Bosscheweg
0,0.000000,0.000977
1,0.061523,0.044922
2,0.004883,0.021484
3,0.057016,0.015024
4,0.005155,0.004179
5,0.000000,0.001953
6,0.009766,0.001953
7,0.135742,0.019531
8,0.017578,0.002930
9,0.000000,0.000000


In [307]:
haha.area_data.loc[haha.area_data["area_name"] == "Bosscheweg"]

,sewer_system,area_name,area_ID,area,geometry,x,y,loc
407,HEU-DRU-BOS,Bosscheweg,294,178900.348383,"POLYGON ((139246.272 411245.098, 139246.45 411...",5.162668,51.690374,"(141, 182)"
555,SMI-DEN-BOS,Bosscheweg,1361,38509.129576,"POLYGON ((153248.442 409009.045, 153241.828 40...",5.359205,51.673278,"(146, 183)"


In [311]:
rain_data

,Start,End,Kwaliteit,Lekerstraat,11. Speelweide/Vlinkert,Rips,Handel,14. Koolhof,13. Heiakker,Vlierden,...,Ooivaarsrijt,de Brand,Bitswijk,de Geer-zuid,De Hoeven(Haarsteeg),11+19 Grevekeur,CHV-terrein,Geb. 12 Vughterpoort,Heusdenseweg(Haarsteeg),Bosscheweg
0,2017-12-31 23:00:00,01-01-2018 00:00:00,HydroNET D2,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,2018-01-01 00:00:00,01-01-2018 01:00:00,HydroNET D2,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0124,0.0000,0.0000,0.0000,0.0000,0.0000
2,2018-01-01 01:00:00,01-01-2018 02:00:00,HydroNET D2,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0244,0.0082,0.0000,0.0789,0.0489,0.0000,0.0000,0.0820,0.0000
3,2018-01-01 02:00:00,01-01-2018 03:00:00,HydroNET D2,0.0000,0.0223,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0186,0.0932,0.0199,0.0689,0.0186,0.0373,0.0351,0.0609,0.0205,0.0000
4,2018-01-01 03:00:00,01-01-2018 04:00:00,HydroNET D2,0.0000,0.0000,0.0000,0.0000,0.0477,0.0049,0.0032,...,0.1992,0.0509,0.0264,0.0984,0.0036,0.0269,0.0110,0.0323,0.0176,0.0000
5,2018-01-01 04:00:00,01-01-2018 05:00:00,HydroNET D2,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0197,0.1833,0.0000,0.0231,0.0672,0.0000,0.0000,0.0000
6,2018-01-01 05:00:00,01-01-2018 06:00:00,HydroNET D2,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.2495,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
7,2018-01-01 06:00:00,01-01-2018 07:00:00,HydroNET D2,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
8,2018-01-01 07:00:00,01-01-2018 08:00:00,HydroNET D2,0.2993,0.0875,0.0000,0.1210,0.0000,0.0000,0.0818,...,0.0186,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
9,2018-01-01 08:00:00,01-01-2018 09:00:00,HydroNET D2,0.1071,0.0631,0.1508,0.5420,0.1091,0.0925,0.0729,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [309]:
cell_index(5.359205, 51.673278)

(146, 183)

In [310]:
cell_index(5.162668, 51.690374)

(141, 182)

In [298]:
hourly_rd.columns

Index(['Lekerstraat', '11. Speelweide/Vlinkert', 'Rips', 'Handel',
       '14. Koolhof', '13. Heiakker', 'Vlierden', 'Helenaveen', 'Heusden',
       'Liender/Loverbosch-noord',
       ...
       'Ooivaarsrijt', 'de Brand', 'Bitswijk', 'de Geer-zuid',
       'De Hoeven(Haarsteeg)', '11+19 Grevekeur', 'CHV-terrein',
       'Geb. 12 Vughterpoort', 'Heusdenseweg(Haarsteeg)', 'Bosscheweg'],
      dtype='object', length=433)

In [280]:
sum(x[x.columns.difference(["Start", "End"])].sum(axis=1) == 0) / (24)

190.16666666666666

In [267]:
x

area_name,Beijerd en 't Riet,30. Akkers I,gebied E; Overlaet E,Geb. 8 Orthenpoort,de Brand,Westerbroek,gebied C1; Hondberg-noord,Moerboom,Spoorstraat,geb. 6c Carolushof,...,08. Stiphout,Wildsedijk,Langenboom,Deursen,Dennenburg,RG Slievenpark,Coudevoort,Bolle Akker,Start,End
0,0.258789,0.003906,0.004883,0.001953,0.002930,0.000000,0.004883,0.023438,0.051758,0.001953,...,0.003906,0.011719,0.127930,0.280273,0.280273,0.132812,0.000000,0.211914,2018-01-02 00:00:00,2018-01-02 01:00:00
1,0.508789,0.164062,0.023438,0.027344,0.030273,0.055664,0.023438,0.434570,0.379883,0.027344,...,0.242188,0.028320,0.213867,0.049805,0.049805,0.029297,0.055664,0.316406,2018-01-02 01:00:00,2018-01-02 02:00:00
2,0.002930,0.130859,0.013672,0.022461,0.016602,0.003906,0.013672,0.009766,0.000000,0.022461,...,0.139648,0.006836,0.028320,0.146484,0.146484,0.373047,0.003906,0.241211,2018-01-02 02:00:00,2018-01-02 03:00:00
3,0.332406,0.156625,0.023813,-0.000601,0.019907,0.064828,0.023813,0.105844,0.352914,-0.000601,...,0.148813,0.037485,0.144907,0.832407,0.832407,0.086313,0.064828,0.169321,2018-01-02 03:00:00,2018-01-02 04:00:00
4,0.330350,0.005155,0.009061,0.026639,0.005155,0.001249,0.009061,0.048124,0.931912,0.026639,...,0.006132,0.034452,0.110624,0.722929,0.722929,0.056913,0.001249,0.030546,2018-01-02 04:00:00,2018-01-02 05:00:00
5,0.013672,0.000000,0.000977,0.000000,0.000977,0.001953,0.000977,0.081055,0.077148,0.000000,...,0.000000,0.000000,0.014648,0.001953,0.001953,0.000000,0.001953,0.000000,2018-01-02 05:00:00,2018-01-02 06:00:00
6,0.000000,0.000000,0.000977,0.007812,0.001953,0.000000,0.000977,0.000000,0.000977,0.007812,...,0.000000,0.001953,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2018-01-02 06:00:00,2018-01-02 07:00:00
7,0.114258,0.001953,0.004883,0.022461,0.012695,0.015625,0.004883,0.007812,0.203125,0.022461,...,0.000000,0.005859,0.013672,0.005859,0.005859,0.009766,0.015625,0.000977,2018-01-02 07:00:00,2018-01-02 08:00:00
8,0.077148,0.070312,0.004883,0.004883,0.000977,0.001953,0.004883,0.167969,0.018555,0.004883,...,0.055664,0.078125,0.226562,0.213867,0.213867,0.055664,0.001953,0.031250,2018-01-02 08:00:00,2018-01-02 09:00:00
9,0.036133,0.066406,0.000000,0.000000,0.000000,0.000000,0.000000,0.078125,0.148438,0.000000,...,0.017578,0.031250,0.007812,0.008789,0.008789,0.139648,0.000000,0.001953,2018-01-02 09:00:00,2018-01-02 10:00:00


In [256]:
min(haha.rain_prediction[1][3].flatten())

-0.00060272216796875

In [203]:
haha.area_data.head()

,sewer_system,area_name,area_ID,area,geometry,x,y,loc
0,CUI-CUI-BEI,Beijerd en 't Riet,316,972233.664525,"POLYGON ((188476.626 415967.071, 188542.047 41...",5.871015,51.724119,"(160, 181)"
1,HEL-HEL-AKI,30. Akkers I,792,373952.057344,"POLYGON ((172555.205 385189.083, 172472.248 38...",5.639523,51.456942,"(153, 192)"
2,HER-ROS-OVE,gebied E; Overlaet E,97,711823.358004,"POLYGON ((153801.956 415030.312, 153801.194 41...",5.357160,51.722117,"(146, 181)"
3,HER-HER-ORP,Geb. 8 Orthenpoort,539,190067.950313,"POLYGON ((149224.757 412105.929, 149224.116 41...",5.301593,51.699136,"(144, 182)"
4,HER-BRA-BRA,de Brand,770,658138.191970,"POLYGON ((153600.725 410510.338, 153375.621 41...",5.358340,51.687064,"(146, 182)"


In [207]:
haha.grid_layers.head()

90000     0
93600     1
97200     2
100800    3
104400    4
dtype: int64

In [222]:
x = haha.area_data["loc"].apply(lambda i: rain_prediction[1][haha.grid_layers, i[0], i[1]])

In [223]:
x.index = haha.area_data["area_name"]

In [231]:
y = x.apply(pd.Series)

In [239]:
pd.Series(haha.grid_layers.index).apply(lambda i: times[0] + datetime.timedelta(seconds = i))

0        2018-01-02 00:00:00
1        2018-01-02 01:00:00
2        2018-01-02 02:00:00
3        2018-01-02 03:00:00
4        2018-01-02 04:00:00
5        2018-01-02 05:00:00
6        2018-01-02 06:00:00
7        2018-01-02 07:00:00
8        2018-01-02 08:00:00
9        2018-01-02 09:00:00
10       2018-01-02 10:00:00
11       2018-01-02 11:00:00
12       2018-01-02 12:00:00
13       2018-01-02 13:00:00
14       2018-01-02 14:00:00
15       2018-01-02 15:00:00
16       2018-01-02 16:00:00
17       2018-01-02 17:00:00
18       2018-01-02 18:00:00
19       2018-01-02 19:00:00
20       2018-01-02 20:00:00
21       2018-01-02 21:00:00
22       2018-01-02 22:00:00
23       2018-01-02 23:00:00
24       2018-01-03 00:00:00
25       2018-01-03 01:00:00
26       2018-01-03 02:00:00
27       2018-01-03 03:00:00
28       2018-01-03 04:00:00
29       2018-01-03 05:00:00
                 ...        
119983   2019-08-07 21:00:00
119984   2019-08-07 21:00:00
119985   2019-08-07 21:00:00
119986   2019-

In [212]:
haha.area_data["loc"][0]

(160, 181)

In [217]:
rain_prediction[1][haha.grid_layers, haha.area_data["loc"][0][0], haha.area_data["loc"][0][1]]

array([0.25878906, 0.50878906, 0.00292969, ..., 0.        , 0.        ,
       0.        ])

In [216]:
rain_prediction[1][0:10,1,0]

array([0.        , 0.        , 0.00390625, 0.00135148, 0.00027239,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [105]:
rain_prediction[1][0][0:10, 0:10]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.00097656, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.00097656, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.       

In [184]:
times = rain_data["Start"].apply(lambda i: i.replace(minute=0))

In [185]:
int_times = times.apply(lambda i: int((i - times[0]).total_seconds()))

In [186]:
int_times

0                0
1             3600
2             7200
3            10800
4            14400
5            18000
6            21600
7            25200
8            28800
9            32400
10           36000
11           39600
12           43200
13           46800
14           50400
15           54000
16           57600
17           61200
18           64800
19           68400
20           72000
21           75600
22           79200
23           82800
24           86400
25           90000
26           93600
27           97200
28          100800
29          104400
            ...   
128072    60991200
128073    60991200
128074    60991200
128075    60991200
128076    60991200
128077    60991200
128078    60994800
128079    60994800
128080    60994800
128081    60994800
128082    60994800
128083    60994800
128084    60994800
128085    60994800
128086    60994800
128087    60994800
128088    60994800
128089    60994800
128090    60998400
128091    60998400
128092    60998400
128093    60

In [187]:
s = pd.Series(range(len(rain_prediction[0])),
              index = rain_prediction[0]["start"].apply(lambda i: int((i - times[0]).total_seconds())))

In [195]:
s

start
90000           0
93600           1
97200           2
100800          3
104400          4
108000          5
111600          6
115200          7
118800          8
122400          9
126000         10
129600         11
133200         12
136800         13
140400         14
144000         15
147600         16
151200         17
154800         18
158400         19
162000         20
165600         21
169200         22
172800         23
176400         24
180000         25
183600         26
187200         27
190800         28
194400         29
            ...  
52340400    14514
52344000    14515
52347600    14516
52351200    14517
52354800    14518
52358400    14519
52362000    14520
52365600    14521
52369200    14522
52372800    14523
52376400    14524
52380000    14525
52383600    14526
52387200    14527
52390800    14528
52394400    14529
52398000    14530
52401600    14531
52405200    14532
52408800    14533
52412400    14534
52416000    14535
52419600    14536
52423200    14537
5242

In [188]:
rain_prediction[0]

,pred,start,end
0,2018-01-01,2018-01-02 00:00:00,2018-01-02 01:00:00
2,2018-01-01,2018-01-02 01:00:00,2018-01-02 02:00:00
3,2018-01-01,2018-01-02 02:00:00,2018-01-02 03:00:00
4,2018-01-01,2018-01-02 03:00:00,2018-01-02 04:00:00
6,2018-01-01,2018-01-02 04:00:00,2018-01-02 05:00:00
7,2018-01-01,2018-01-02 05:00:00,2018-01-02 06:00:00
8,2018-01-01,2018-01-02 06:00:00,2018-01-02 07:00:00
9,2018-01-01,2018-01-02 07:00:00,2018-01-02 08:00:00
11,2018-01-01,2018-01-02 08:00:00,2018-01-02 09:00:00
12,2018-01-01,2018-01-02 09:00:00,2018-01-02 10:00:00


In [199]:
s.loc[s.index.intersection(int_times)]

90000           0
93600           1
97200           2
100800          3
104400          4
108000          5
111600          6
115200          7
118800          8
122400          9
126000         10
129600         11
133200         12
136800         13
140400         14
144000         15
147600         16
151200         17
154800         18
158400         19
162000         20
165600         21
169200         22
172800         23
176400         24
180000         25
183600         26
187200         27
190800         28
194400         29
            ...  
50450400    13989
50450400    13989
50450400    13989
50450400    13989
50450400    13989
50450400    13989
50454000    13990
50454000    13990
50454000    13990
50454000    13990
50454000    13990
50454000    13990
50454000    13990
50454000    13990
50454000    13990
50454000    13990
50454000    13990
50454000    13990
50457600    13991
50457600    13991
50457600    13991
50457600    13991
50457600    13991
50457600    13991
50457600  

In [198]:
s.reindex(index=int_times)

Start
0           NaN
3600        NaN
7200        NaN
10800       NaN
14400       NaN
18000       NaN
21600       NaN
25200       NaN
28800       NaN
32400       NaN
36000       NaN
39600       NaN
43200       NaN
46800       NaN
50400       NaN
54000       NaN
57600       NaN
61200       NaN
64800       NaN
68400       NaN
72000       NaN
75600       NaN
79200       NaN
82800       NaN
86400       NaN
90000       0.0
93600       1.0
97200       2.0
100800      3.0
104400      4.0
           ... 
60991200    NaN
60991200    NaN
60991200    NaN
60991200    NaN
60991200    NaN
60991200    NaN
60994800    NaN
60994800    NaN
60994800    NaN
60994800    NaN
60994800    NaN
60994800    NaN
60994800    NaN
60994800    NaN
60994800    NaN
60994800    NaN
60994800    NaN
60994800    NaN
60998400    NaN
60998400    NaN
60998400    NaN
60998400    NaN
60998400    NaN
60998400    NaN
60998400    NaN
60998400    NaN
60998400    NaN
60998400    NaN
60998400    NaN
60998400    NaN
Length: 128102, dt

In [171]:
s

start
90000           0
90000           1
93600           2
97200           3
100800          4
100800          5
104400          6
108000          7
111600          8
115200          9
115200         10
118800         11
122400         12
126000         13
129600         14
133200         15
133200         16
136800         17
136800         18
140400         19
144000         20
147600         21
147600         22
151200         23
151200         24
154800         25
154800         26
158400         27
158400         28
162000         29
            ...  
52340400    14534
52344000    14535
52347600    14536
52351200    14537
52354800    14538
52358400    14539
52362000    14540
52365600    14541
52369200    14542
52372800    14543
52376400    14544
52380000    14545
52383600    14546
52387200    14547
52390800    14548
52394400    14549
52398000    14550
52401600    14551
52405200    14552
52408800    14553
52412400    14554
52416000    14555
52419600    14556
52423200    14557
5242